In [1]:
import gradio as gr
import pandas as pd
import re
import PyPDF2 
import numpy as np
import matplotlib.pyplot as plt
from experta import *
import nbimporter
import ML
%run KBS.ipynb
%run sentiment_analysis.ipynb

# The variables dictionary and other relevant functions
variables = {
    "net income": ["net income", "net earnings","net income loss"],
    "total assets": ["total assets"],
    "total stockholders’ equity": ["total stockholders’ equity", "total equity","total stockholder's equity"],
    "total current assets": ["total current assets","total cash, cash equivalents, restricted cash and restricted cash equivalents"],
    "cash and cash equivalents": ["cash and cash equivalents", "cash & equivalents"],
    "retained earnings": ["retained earnings"],
    "short-term investments": ["short-term investments"],
    "accounts receivable, net": ["accounts receivable, net", "accounts receivable","total receivables, net"],
    "total current liabilities": ["total current liabilities","total debt"],
    "debt and finance leases, net of current portion": ["debt and finance leases, net of current portion", "long-term debt"],
    "interest expense": ["interest expense"],
    "interest income": ["interest income"],
    "total liabilities": ["total liabilities"],
    "total debt": ["total debt"],
    "equity": ["equity"],
    "operating profit": ["income from operations","operating profit", "operating income","operating activities net income","ebit"],
    "long-term liabilities": ["long-term liabilities"],
    "total costs and expenses": ["total costs and expenses","total expenses","total cost of revenues"],
    "accounts receivable": ["accounts receivable","accounts receivable, net","accounts receivable","total receivables, net"],
    "marketable securities": ["marketable securities","short-term investments"],
    "inventory": ["inventory"],
    "total revenues": ["total revenues"],
    "gross profit":["gross profit"],
    "operating income":["income from operations"],
    "tax rate":["effective tax rate was","effective tax rate","tax rate"],
}

patterns = [
    (r'\n', ' '),              # Replace newlines with spaces
    (r'[$.()%:?;<>{}•!]', ' '), # Remove special characters
    (r'\s+', ' ') ,   # Replace multiple spaces with a single space 
]

def clean_text(text, patterns):
    text = text.lower()  # Convert text to lowercase
    for pattern, replacement in patterns:
        text = re.sub(pattern, replacement, text)
    return text

def extract_values(text, term):
    term_escaped = re.escape(term)
    # Modified pattern to match numbers with or without commas
    pattern = rf'{term_escaped}\s+(\d+(?:,\d+)|\d+)\s+(\d+(?:,\d+)|\d+)'
    matches = re.findall(pattern, text)
    return matches

def convert_to_float(number_str):
    return float(number_str.replace(',', ''))


In [2]:
import pandas as pd
import PyPDF2
import re

financial_dict = {}

def check_for_sentences(text, keywords):
    """
    Check if the text contains any of the given keywords.
    """
    # Convert text to lowercase to ensure case-insensitive comparison
    text = text.lower()
    # Check if any keyword is in the text
    return any(keyword.lower() in text for keyword in keywords)

def process_file(file):
    reader = PyPDF2.PdfReader(file)
    num_pages = len(reader.pages)
    text = []

    # Extract text from each page
    for page_num in range(num_pages):
        page = reader.pages[page_num]
        text.append(page.extract_text())
    
    # Clean and preprocess the text
    for i in range(len(text)):
        text[i] = clean_text(text[i], patterns)  # Your original clean_text function

    # Aggregate all text for sentiment analysis
    full_text = " ".join(text)
    
    
    is_valid_pdf = check_for_sentences(full_text, ["form 10-q","(Exact name of registrant as specified in its charter)"])
    print("Is 10-Q Form:", is_valid_pdf)
    if not is_valid_pdf:
        return ("The uploaded PDF is not a valid 10-Q form report.",) + (None,) * 38
    
    
    # Calculate sentiment using both methods
    lm_score1, lm_score2, lm_sentiment = calculate_sentiment(preprocess_text(full_text))
    finbert_pos, finbert_neg, finbert_neu, finbert_sentiment = get_finbert_sentiment(full_text)
    
    # Your original financial processing logic
    for page in reversed(text):
        for var, keys in variables.items():
            for key in keys:
                matches = extract_values(page, key)
                if matches:
                    financial_dict[var] = [convert_to_float(matches[0][0]), convert_to_float(matches[0][1])]
                    break

    # Set variables not found to one
    for var in variables:
        if var not in financial_dict:
            financial_dict[var] = [1, 1]

    # Prepare the output string for financial data
    output = "\nExtracted Financial Data:\n"
    for key, value in financial_dict.items():
        output += f"{key}: Current Year = {value[0]}, Previous Year = {value[1]}\n"
    
    # Run your financial analysis logic (unchanged)
    results = run_combined_scores(financial_dict)
       
    # Extract and display scores
    beneish_score = results["BeneishMScore"]
    piotroski_score = results["PiotroskiFScore"]
    springate_score = results["SpringateScore"]
    profit_ability_analysis = results["ProfitabilityAnalysis"]
    solvency_analysis = results["SolvencyAnalysis"]
    financial_ratios = results["FinancialRatios"]
    liquidity_analysis = results["LiquidityAnalysis"]
 

    # Capture and store the plots
    plot_profit_outputs = [
        plot_comparison(metric, values["Current Year"], values["Previous Year"])
        for metric, values in profit_ability_analysis.items()
        if "Current Year" in values and "Previous Year" in values
    ]

    plot_solvency_outputs = [
        plot_comparison(metric, values["Current Year"], values["Previous Year"])
        for metric, values in solvency_analysis.items()
        if "Current Year" in values and "Previous Year" in values
    ]
    profitability_analysis = results["ProfitabilityAnalysis"]

        # Create text descriptions and corresponding plots for each metric
    roa_text = f"ROA:\nCurrent Year = {profitability_analysis['ROA']['Current Year']:.4f}\nPrevious Year = {profitability_analysis['ROA']['Previous Year']:.4f}\nChange = {profitability_analysis['ROA']['Change']:.2f}%\nExplanation: {profitability_analysis['ROA']['Change Explanation']}"
    roa_chart = plot_comparison("ROA", profitability_analysis['ROA']['Current Year'], profitability_analysis['ROA']['Previous Year'])

    roe_text = f"ROE:\nCurrent Year = {profitability_analysis['ROE']['Current Year']:.4f}\nPrevious Year = {profitability_analysis['ROE']['Previous Year']:.4f}\nChange = {profitability_analysis['ROE']['Change']:.2f}%\nExplanation: {profitability_analysis['ROE']['Change Explanation']}"
    roe_chart = plot_comparison("ROE", profitability_analysis['ROE']['Current Year'], profitability_analysis['ROE']['Previous Year'])

    opm_text = f"Operating Profit Margin:\nCurrent Year = {profitability_analysis['Operating Profit Margin']['Current Year']:.4f}\nPrevious Year = {profitability_analysis['Operating Profit Margin']['Previous Year']:.4f}\nChange = {profitability_analysis['Operating Profit Margin']['Change']:.2f}%\nExplanation: {profitability_analysis['Operating Profit Margin']['Change Explanation']}"
    opm_chart = plot_comparison("Operating Profit Margin", profitability_analysis['Operating Profit Margin']['Current Year'], profitability_analysis['Operating Profit Margin']['Previous Year'])

    npm_text = f"Net Profit Margin:\nCurrent Year = {profitability_analysis['Net Profit Margin']['Current Year']:.4f}\nPrevious Year = {profitability_analysis['Net Profit Margin']['Previous Year']:.4f}\nChange = {profitability_analysis['Net Profit Margin']['Change']:.2f}%\nExplanation: {profitability_analysis['Net Profit Margin']['Change Explanation']}"
    npm_chart = plot_comparison("Net Profit Margin", profitability_analysis['Net Profit Margin']['Current Year'], profitability_analysis['Net Profit Margin']['Previous Year'])

    gross_margin_text = f"Gross Margin:\nCurrent Year = {profitability_analysis['Gross Margin']['Current Year']:.4f}\nPrevious Year = {profitability_analysis['Gross Margin']['Previous Year']:.4f}\nChange = {profitability_analysis['Gross Margin']['Change']:.2f}%\nExplanation: {profitability_analysis['Gross Margin']['Change Explanation']}"
    gross_margin_chart = plot_comparison("Gross Margin", profitability_analysis['Gross Margin']['Current Year'], profitability_analysis['Gross Margin']['Previous Year'])

    # Overall interpretation text
    overall_interpretation_profit = profitability_analysis["Overall Interpretation"]
    
        # Solvency analysis
    debt_to_equity_text = f"Debt-to-Equity Ratio:\nCurrent Year = {solvency_analysis['Debt-to-Equity Ratio']['Current Year']:.4f}\nPrevious Year = {solvency_analysis['Debt-to-Equity Ratio']['Previous Year']:.4f}\nChange = {solvency_analysis['Debt-to-Equity Ratio']['Change']:.2f}%\nExplanation: {solvency_analysis['Debt-to-Equity Ratio']['Change Explanation']}"
    debt_to_equity_chart = plot_comparison("Debt-to-Equity Ratio", solvency_analysis['Debt-to-Equity Ratio']['Current Year'], solvency_analysis['Debt-to-Equity Ratio']['Previous Year'])

    equity_ratio_text = f"Equity Ratio:\nCurrent Year = {solvency_analysis['Equity Ratio']['Current Year']:.4f}\nPrevious Year = {solvency_analysis['Equity Ratio']['Previous Year']:.4f}\nChange = {solvency_analysis['Equity Ratio']['Change']:.2f}%\nExplanation: {solvency_analysis['Equity Ratio']['Change Explanation']}"
    equity_ratio_chart = plot_comparison("Equity Ratio", solvency_analysis['Equity Ratio']['Current Year'], solvency_analysis['Equity Ratio']['Previous Year'])

    liabilities_to_assets_text = f"Total Liabilities/Total Assets:\nCurrent Year = {solvency_analysis['Total Liabilities/Total Assets']['Current Year']:.4f}\nPrevious Year = {solvency_analysis['Total Liabilities/Total Assets']['Previous Year']:.4f}\nChange = {solvency_analysis['Total Liabilities/Total Assets']['Change']:.2f}%\nExplanation: {solvency_analysis['Total Liabilities/Total Assets']['Change Explanation']}"
    liabilities_to_assets_chart = plot_comparison("Total Liabilities/Total Assets", solvency_analysis['Total Liabilities/Total Assets']['Current Year'], solvency_analysis['Total Liabilities/Total Assets']['Previous Year'])

    overall_interpretation_solvency = solvency_analysis["Overall Interpretation"]
    
    # Generate pie chart
    pie_chart = plot_f_score_components_pie(piotroski_score)
    
    user_beneish_score=interpret_beneish_score(beneish_score)
    
    user_piotroski_score=interpret_piotroski_score(piotroski_score)
    
    springate_score=interpret_springate_score(springate_score)
        
    sentiment_interpretation = interpret_sentiment(lm_sentiment, finbert_sentiment)
    # Generate PDF
    pdf_file_path = "financial_analysis_report.pdf"
    create_pdf(pdf_file_path, (
        financial_dict, user_beneish_score, user_piotroski_score, pie_chart,
        springate_score, roa_text, roa_chart, roe_text, roe_chart, opm_text,
        opm_chart, npm_text, npm_chart, gross_margin_text, gross_margin_chart,
        overall_interpretation_profit, debt_to_equity_text, debt_to_equity_chart,
        equity_ratio_text, equity_ratio_chart, liabilities_to_assets_text,
        liabilities_to_assets_chart, overall_interpretation_solvency,
        sentiment_interpretation, financial_ratios
    ))

    
        # Liquidity Analysis
    liquidity_analysis = results["LiquidityAnalysis"]
    
    # Create text descriptions and corresponding plots for each liquidity metric
    interest_coverage_text = f"Interest Coverage Ratio:\nCurrent Year = {liquidity_analysis['Interest Coverage Ratio']['current']:.4f}\nPrevious Year = {liquidity_analysis['Interest Coverage Ratio']['previous']:.4f}\nExplanation: {liquidity_analysis['Interest Coverage Ratio']['explanation']}"
    interest_coverage_chart = plot_comparison("Interest Coverage Ratio", liquidity_analysis['Interest Coverage Ratio']['current'], liquidity_analysis['Interest Coverage Ratio']['previous'])
    
    quick_ratio_text = f"Quick Ratio:\nCurrent Year = {liquidity_analysis['Quick Ratio']['current']:.4f}\nPrevious Year = {liquidity_analysis['Quick Ratio']['previous']:.4f}\nExplanation: {liquidity_analysis['Quick Ratio']['explanation']}"
    quick_ratio_chart = plot_comparison("Quick Ratio", liquidity_analysis['Quick Ratio']['current'], liquidity_analysis['Quick Ratio']['previous'])
    
    inventory_turnover_text = f"Inventory Turnover Ratio:\nCurrent Year = {liquidity_analysis['Inventory Turnover Ratio']['current']:.4f}\nPrevious Year = {liquidity_analysis['Inventory Turnover Ratio']['previous']:.4f}\nExplanation: {liquidity_analysis['Inventory Turnover Ratio']['explanation']}"
    inventory_turnover_chart = plot_comparison("Inventory Turnover Ratio", liquidity_analysis['Inventory Turnover Ratio']['current'], liquidity_analysis['Inventory Turnover Ratio']['previous'])
    
    cash_ratio_text = f"Cash Ratio:\nCurrent Year = {liquidity_analysis['Cash Ratio']['current']:.4f}\nPrevious Year = {liquidity_analysis['Cash Ratio']['previous']:.4f}\nExplanation: {liquidity_analysis['Cash Ratio']['explanation']}"
    cash_ratio_chart = plot_comparison("Cash Ratio", liquidity_analysis['Cash Ratio']['current'], liquidity_analysis['Cash Ratio']['previous'])
    
    current_ratio_text = f"Current Ratio:\nCurrent Year = {liquidity_analysis['Current Ratio']['current']:.4f}\nPrevious Year = {liquidity_analysis['Current Ratio']['previous']:.4f}\nExplanation: {liquidity_analysis['Current Ratio']['explanation']}"
    current_ratio_chart = plot_comparison("Current Ratio", liquidity_analysis['Current Ratio']['current'], liquidity_analysis['Current Ratio']['previous'])

    financial_ratios_table = format_table_for_ratio(financial_ratios)
    financial_dict_table = format_table_for_data(financial_dict)
    prediction = ML.predict_financial_status(financial_ratios)
    prediction_result = prediction_result_function(prediction)
    
    # Update the return statement to include the new liquidity analysis outputs
    return (
        'The file is valid for processing',
        roa_text,                  # 1. ROA
        roa_chart,                 # 2. ROA Chart
        roe_text,                  # 3. ROE
        roe_chart,                 # 4. ROE Chart
        opm_text,                  # 5. Operating Profit Margin
        opm_chart,                 # 6. Operating Profit Margin Chart
        npm_text,                  # 7. Net Profit Margin
        npm_chart,                 # 8. Net Profit Margin Chart
        gross_margin_text,         # 9. Gross Margin
        gross_margin_chart,        # 10. Gross Margin Chart
        overall_interpretation_profit, # 11. Overall Profitability Interpretation
    
        # Solvency Metrics
        debt_to_equity_text,       # 12. Debt-to-Equity Ratio
        debt_to_equity_chart,      # 13. Debt-to-Equity Ratio Chart
        equity_ratio_text,         # 14. Equity Ratio
        equity_ratio_chart,        # 15. Equity Ratio Chart
        liabilities_to_assets_text, # 16. Total Liabilities/Total Assets
        liabilities_to_assets_chart, # 17. Total Liabilities/Total Assets Chart
        overall_interpretation_solvency, # 18. Overall Solvency Interpretation
        
        # Liquidity Metrics
        interest_coverage_text,    # 19. Interest Coverage Ratio
        interest_coverage_chart,   # 20. Interest Coverage Ratio Chart 
        quick_ratio_text,          # 21. Quick Ratio
        quick_ratio_chart,         # 22. Quick Ratio Chart
        inventory_turnover_text,   # 23. Inventory Turnover Ratio
        inventory_turnover_chart,  # 24. Inventory Turnover Ratio Chart
        cash_ratio_text,           # 25. Cash Ratio
        cash_ratio_chart,          # 26. Cash Ratio Chart
        current_ratio_text,        # 27. Current Ratio
        current_ratio_chart,       # 28. Current Ratio Chart

        user_piotroski_score,      # 29. Financial Health (Piotroski F-Score)
        pie_chart,                 # 30. Piotroski Components

        prediction_result,
        sentiment_interpretation,  # 31. Sentiment Interpretation
        springate_score,           # 32. Company State (Springate Score)
        user_beneish_score,        # 33. Earnings Manipulation (Beneish M-Score)
        # financial_ratios,          # 34. Financial Ratios
        # financial_dict,            # 35. Extracted Financial Data
        
        financial_dict_table,      # 36. Financial Data Table
        financial_ratios_table,    # 37. Financial Ratios Table
    )

In [3]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image
from reportlab.lib.units import inch
import io
from PIL import Image as PilImage

def prediction_result_function(khaled):
    if khaled == 0 :
        return "good"
    else : 
        return "Bankrupt"

def image_to_bytes(image_obj):
    """Convert a PIL image object to a bytes-like object."""
    with io.BytesIO() as buffer:
        image_obj.save(buffer, format="PNG")
        return buffer.getvalue()

from reportlab.platypus import Paragraph
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Image

def create_pdf(file_path, data):
    doc = SimpleDocTemplate(file_path, pagesize=letter)
    story = []

    styles = getSampleStyleSheet()
    normal = styles['Normal']
    heading = styles['Heading1']
    heading2 = styles['Heading2']
    
    (
        financial_dict, user_beneish_score, user_piotroski_score, pie_chart,
        springate_score, roa_text, roa_chart, roe_text, roe_chart, opm_text,
        opm_chart, npm_text, npm_chart, gross_margin_text, gross_margin_chart,
        overall_interpretation_profit, debt_to_equity_text, debt_to_equity_chart,
        equity_ratio_text, equity_ratio_chart, liabilities_to_assets_text,
        liabilities_to_assets_chart, overall_interpretation_solvency,
        sentiment_interpretation, financial_ratios
    ) = data

    # Add extracted financial data
    story.append(Paragraph("Extracted Financial Data", heading))
    for key, value in financial_dict.items():
        story.append(Paragraph(f"{key}: Current Year = {value[0]}, Previous Year = {value[1]}", normal))
    
    # Add other sections with text
    story.append(Paragraph("Earnings Manipulation", heading2))
    story.append(Paragraph(user_beneish_score, normal))
    
    story.append(Paragraph("Financial Health", heading2))
    story.append(Paragraph(user_piotroski_score, normal))
    
    story.append(Paragraph("Piotroski Components", heading2))
    if isinstance(pie_chart, PilImage.Image):
        pie_chart_bytes = image_to_bytes(pie_chart)
        story.append(Image(io.BytesIO(pie_chart_bytes), width=4*inch, height=3*inch))
    
    story.append(Paragraph("Company State", heading2))
    story.append(Paragraph(springate_score, normal))
    
    # Profitability Metrics
    story.append(Paragraph("Profitability Metrics", heading))
    for text, chart in [
        (roa_text, roa_chart), (roe_text, roe_chart),
        (opm_text, opm_chart), (npm_text, npm_chart),
        (gross_margin_text, gross_margin_chart)
    ]:
        story.append(Paragraph(text, heading2))
        if isinstance(chart, PilImage.Image):
            chart_bytes = image_to_bytes(chart)
            story.append(Image(io.BytesIO(chart_bytes), width=4*inch, height=3*inch))

    story.append(Paragraph("Overall Interpretation of Profitability", heading2))
    story.append(Paragraph(overall_interpretation_profit, normal))

    # Solvency Metrics
    story.append(Paragraph("Solvency Metrics", heading))
    for text, chart in [
        (debt_to_equity_text, debt_to_equity_chart), (equity_ratio_text, equity_ratio_chart),
        (liabilities_to_assets_text, liabilities_to_assets_chart)
    ]:
        story.append(Paragraph(text, heading2))
        if isinstance(chart, PilImage.Image):
            chart_bytes = image_to_bytes(chart)
            story.append(Image(io.BytesIO(chart_bytes), width=4*inch, height=3*inch))
    
    story.append(Paragraph("Overall Interpretation of Solvency", heading2))
    story.append(Paragraph(overall_interpretation_solvency, normal))
    
    # Sentiment Interpretation
    story.append(Paragraph("Sentiment Analysis", heading))
    story.append(Paragraph(sentiment_interpretation, normal))
    
    # Financial Ratios
    story.append(Paragraph("Financial Ratios", heading2))
    
    # Ensure financial_ratios is a string
    if isinstance(financial_ratios, dict):
        # Convert dictionary to string
        financial_ratios_str = "\n".join(f"{key}: {value}" for key, value in financial_ratios.items())
    else:
        financial_ratios_str = financial_ratios
    
    story.append(Paragraph(financial_ratios_str, normal))

    # Build the PDF
    doc.build(story)


In [4]:
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image as PilImage

def plot_f_score_components_pie(piotroski_score):
    components = {
        'Net Income': piotroski_score.get('NI', 0),
        'ROA': piotroski_score.get('ROA', 0),
        'CFO': piotroski_score.get('CFO', 0),
        'Accrual': piotroski_score.get('Accrual', 0),
        'Leverage': piotroski_score.get('Leverage', 0),
        'Liquidity': piotroski_score.get('Liquidity', 0),
        'Dilution': piotroski_score.get('Dilution', 0),
        'Gross Margin': piotroski_score.get('Gross Margin', 0),
        'Asset Turnover': piotroski_score.get('Asset Turnover', 0)
    }
    
    # Prepare labels and sizes for equal sections
    labels = list(components.keys())
    sizes = [1] * len(labels)  # Equal size for each segment
    
    # Define colors
    colors = ['white' if value == 0 else color for value, color in zip(components.values(), ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#c2c2f0', '#ffb3e6', '#c2f0c2', '#ff6666', '#c2c2c2'])]
    # Plot the pie chart
    plt.figure(figsize=(10, 8))
    plt.pie(sizes, labels=labels, colors=colors, startangle=140, wedgeprops={'edgecolor': (0.8,0.8, 0.8)}, autopct='')
    plt.title('Piotroski F-Score Components')

    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    plt.close()
    buffer.seek(0)

    # Convert BytesIO to PIL Image
    image = PilImage.open(buffer)
    return image


In [5]:
# Function to format the data into a table
def format_table_for_data(data):
    table = "<table style='width:100%; border-collapse: collapse;'>"
    table += "<tr><th style='border: 1px solid black; padding: 8px;'>Financial Data</th><th style='border: 1px solid black; padding: 8px;'>Current Year</th><th style='border: 1px solid black; padding: 8px;'>Previous Year</th></tr>"
    
    for key, value in data.items():
        if isinstance(value, list) and len(value) == 2:
            current_year_value, previous_year_value = value
            table += (f"<tr><td style='border: 1px solid black; padding: 8px;'>{key}</td>"
                      f"<td style='border: 1px solid black; padding: 8px;'>{current_year_value:.4f}</td>"
                      f"<td style='border: 1px solid black; padding: 8px;'>{previous_year_value:.4f}</td></tr>")
        else:
            # Handle cases where the value is not a list of two items
            table += (f"<tr><td style='border: 1px solid black; padding: 8px;'>{key}</td>"
                      f"<td colspan='2' style='border: 1px solid black; padding: 8px;'>{value}</td></tr>")
    
    table += "</table>"
    return table

# Function to format the data into a table
def format_table_for_ratio(data):
    table = "<table style='width:100%; border-collapse: collapse;'>"
    table += "<tr><th style='border: 1px solid black; padding: 8px;'>Financial Ratio</th><th style='border: 1px solid black; padding: 8px;'>Current Year</th>"
    
    for key, value in data.items():
        if isinstance(value, list) and len(value) == 2:
            current_year_value, previous_year_value = value
            table += (f"<tr><td style='border: 1px solid black; padding: 8px;'>{key}</td>"
                      f"<td style='border: 1px solid black; padding: 8px;'>{current_year_value:.4f}</td>"
                      f"<td style='border: 1px solid black; padding: 8px;'>{previous_year_value:.4f}</td></tr>")
        else:
            # Handle cases where the value is not a list of two items
            table += (f"<tr><td style='border: 1px solid black; padding: 8px;'>{key}</td>"
                      f"<td colspan='2' style='border: 1px solid black; padding: 8px;'>{value}</td></tr>")
    
    table += "</table>"
    return table

In [6]:
import matplotlib.pyplot as plt


from PIL import Image as PilImage
import io


def plot_comparison(metric_name, current_value, previous_value):
    # Plot the values for the current and previous years
    plt.figure(figsize=(6, 4))  # Adjusted size for better visibility
    years = ['Previous Year', 'Current Year']
    values = [previous_value, current_value]
    
    # Create a bar chart
    plt.bar(years, values, color=['blue', 'green'])
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.title(f'{metric_name} Comparison')
    plt.ylim(0, max(values) * 1.2)  # Extend y-axis to make room for labels
    
    # Annotate the bars with the exact values
    plt.text(0, previous_value, f'{previous_value:.2%}', ha='center', va='bottom')
    plt.text(1, current_value, f'{current_value:.2%}', ha='center', va='bottom')

    # Save the plot to a BytesIO object
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    plt.close()
    buffer.seek(0)

    # Convert BytesIO to PIL Image
    image = PilImage.open(buffer)
    return image


In [7]:
import gradio as gr

with gr.Blocks(title="PDF Financial Data Extraction and Ratio Calculation") as demo:

    with gr.Row():
        file_input = gr.File(label="Upload a PDF File")
#     if(ispdf!=''):    
    ispdf = gr.Textbox(label="")

    # Profitability Metrics
    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Profitability Analysis.
    </div>
    """)
    with gr.Row():
        with gr.Column():
            roa_textbox = gr.Textbox(label="ROA")
            roa_image = gr.Image(label="ROA Chart")
        with gr.Column():
            roe_textbox = gr.Textbox(label="ROE")
            roe_image = gr.Image(label="ROE Chart")
        
    with gr.Row():
        with gr.Column():
            opm_textbox = gr.Textbox(label="Operating Profit Margin")
            opm_image = gr.Image(label="Operating Profit Margin Chart") 
        with gr.Column():
            npm_textbox = gr.Textbox(label="Net Profit Margin")
            npm_image = gr.Image(label="Net Profit Margin Chart")
            
    with gr.Row():
        with gr.Column(elem_id="centered-row"):
            gross_margin_textbox = gr.Textbox(label="Gross Margin")
            gross_margin_image = gr.Image(label="Gross Margin Chart")
            
    with gr.Row():
        overall_profitability_interpretation = gr.Textbox(label="Overall Profitability Interpretation")
            

    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Solvency Analysis.
    </div>
    """)
    with gr.Row():
        with gr.Column():
            debt_to_equity_textbox = gr.Textbox(label="Debt-to-Equity Ratio")
            debt_to_equity_image = gr.Image(label="Debt-to-Equity Ratio Chart")
        with gr.Column():
            equity_ratio_textbox = gr.Textbox(label="Equity Ratio")
            equity_ratio_image = gr.Image(label="Equity Ratio Chart")

        

    with gr.Row():
        with gr.Column():
            liabilities_to_assets_textbox = gr.Textbox(label="Total Liabilities/Total Assets")
            liabilities_to_assets_image = gr.Image(label="Total Liabilities/Total Assets Chart")
            
    with gr.Row():
            overall_solvency_interpretation = gr.Textbox(label="Overall Solvency Interpretation")


    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Liquidity Analysis.
    </div>
    """)
    with gr.Row():
        with gr.Column():
            interest_coverage_textbox = gr.Textbox(label="Interest Coverage Ratio")
            interest_coverage_image = gr.Image(label="Interest Coverage Ratio Chart")
        with gr.Column():
            quick_ratio_textbox = gr.Textbox(label="Quick Ratio")
            quick_ratio_image = gr.Image(label="Quick Ratio Chart")

    with gr.Row():
        with gr.Column():
            inventory_turnover_textbox = gr.Textbox(label="Inventory Turnover Ratio")
            inventory_turnover_image = gr.Image(label="Inventory Turnover Ratio Chart")
        with gr.Column():
            cash_ratio_textbox = gr.Textbox(label="Cash Ratio")
            cash_ratio_image = gr.Image(label="Cash Ratio Chart")
            
            
    with gr.Row():
        with gr.Column():
            current_ratio_textbox = gr.Textbox(label="Current Ratio")
            current_ratio_image = gr.Image(label="Current Ratio Chart")

    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Financial Health Analysis.
    </div>
    """)

    with gr.Row():
        with gr.Column():
            financial_health_textbox = gr.Textbox(label="Financial Health")
            piotroski_components_image = gr.Image(label="Piotroski Components")

    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Machine learning prediction for bankrupt.
    </div>
    """)
    with gr.Row():
        # Sentiment Analysis and Financial Ratios
       prediction_textbox = gr.Textbox(label="prediction Analysis")

    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Sentiment Analysis.
    </div>
    """)
    with gr.Row():
        # Sentiment Analysis and Financial Ratios
        sentiment_analysis_html = gr.HTML(label="Sentiment Analysis")

    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Company State Analysis.
    </div>
    """)

    with gr.Row():
            company_state_textbox = gr.Textbox(label="Company State")
            earnings_manipulation_textbox = gr.Textbox(label="Earnings Manipulation")
        
    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
        Financial Data Table.
    </div>
    """)
    with gr.Row():
        financial_data_table_html = gr.HTML(label="Financial Data Table")

    gr.Markdown("""
    <div style="text-align: center; font-size: 40px;">
       Financial Ratio Table
    </div>
    """)
    with gr.Row():
        financial_ratio_table_html = gr.HTML(label="Financial Ratio Table")

    # Event handler to process the file and update outputs
    file_input.change(
        fn=process_file,
        inputs=file_input,
        outputs=[
            ispdf,roa_textbox, roa_image, roe_textbox, roe_image, opm_textbox, opm_image,
            npm_textbox, npm_image, gross_margin_textbox, gross_margin_image, overall_profitability_interpretation,
            debt_to_equity_textbox, debt_to_equity_image, equity_ratio_textbox, equity_ratio_image,
            liabilities_to_assets_textbox, liabilities_to_assets_image, overall_solvency_interpretation,
            interest_coverage_textbox, interest_coverage_image, quick_ratio_textbox, quick_ratio_image,
            inventory_turnover_textbox, inventory_turnover_image, cash_ratio_textbox, cash_ratio_image,
            current_ratio_textbox, current_ratio_image, financial_health_textbox, piotroski_components_image,
            prediction_textbox,
            sentiment_analysis_html, company_state_textbox, earnings_manipulation_textbox,
            # financial_ratios_textbox,
            # extracted_financial_data_textbox,
            financial_data_table_html,
            financial_ratio_table_html,
            
        ]
    )

# Add custom CSS to center the column in the row
demo.css = """
#centered-row {
    display: flex;
    justify-content: center;
    align-items: center;
}
"""

demo.launch(inline=True)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Is 10-Q Form: True
Positive words count: 167
Negative words count: 279
Total words count: 13181
Loughran-McDonald Score 1: -0.0085
Loughran-McDonald Score 2: -0.2511
Final Sentiment: Positive
Is 10-Q Form: True
Positive words count: 116
Negative words count: 256
Total words count: 10331
Loughran-McDonald Score 1: -0.0136
Loughran-McDonald Score 2: -0.3763
Final Sentiment: Positive
